# 1. General Settings

### 1.1 Import libraries

In [1]:
### bugs for transformers(4.28.1), update to newest version 4.31.0
from dataclasses import dataclass, field
from typing import Optional
import torch
import re
import numpy as np
import torch
import random
from accelerate import Accelerator
from datasets import load_dataset
import pandas as pd

from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

[2023-09-04 23:00:04,112] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### 1.2 Random Seed

In [2]:
def set_seed(seed: int):
    """
   A function to fix random seed in `numpy`,`random`,  and `torch`.

    Args:
        seed (`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(1000)

### 1.3 Metrics

In [49]:
# Reference: https://www.nltk.org/

import operator
from functools import reduce
from math import fabs
from random import shuffle

try:
    from scipy.stats.stats import betai
except ImportError:
    betai = None

from nltk.util import LazyConcatenation, LazyMap



def accuracy(reference, test):
    """
    Given a list of reference values and a corresponding list of test
    values, return the fraction of corresponding values that are
    equal.  In particular, return the fraction of indices
    ``0<i<=len(test)`` such that ``test[i] == reference[i]``.

    :type reference: list
    :param reference: An ordered list of reference values.
    :type test: list
    :param test: A list of values to compare against the corresponding
        reference values.
    :raise ValueError: If ``reference`` and ``length`` do not have the
        same length.
    """
    if len(reference) != len(test):
        raise ValueError("Lists must have the same length.")
    return sum(float(x) == float(y) for x, y in zip(reference, test) ) / len(test)



def precision(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of test values that appear in the reference set.
    In particular, return card(``reference`` intersection ``test``)/card(``test``).
    If ``test`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(test) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(test)



def recall(reference, test):
    """
    Given a set of reference values and a set of test values, return
    the fraction of reference values that appear in the test set.
    In particular, return card(``reference`` intersection ``test``)/card(``reference``).
    If ``reference`` is empty, then return None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
        raise TypeError("reference and test should be sets")

    if len(reference) == 0:
        return None
    else:
        return len(reference.intersection(test)) / len(reference)



def f_measure(reference, test, alpha=0.5):
    """
    Given a set of reference values and a set of test values, return
    the f-measure of the test values, when compared against the
    reference values.  The f-measure is the harmonic mean of the
    ``precision`` and ``recall``, weighted by ``alpha``.  In particular,
    given the precision *p* and recall *r* defined by:

    - *p* = card(``reference`` intersection ``test``)/card(``test``)
    - *r* = card(``reference`` intersection ``test``)/card(``reference``)

    The f-measure is:

    - *1/(alpha/p + (1-alpha)/r)*

    If either ``reference`` or ``test`` is empty, then ``f_measure``
    returns None.

    :type reference: set
    :param reference: A set of reference values.
    :type test: set
    :param test: A set of values to compare against the reference set.
    :rtype: float or None
    """
    p = precision(reference, test)
    r = recall(reference, test)
    if p is None or r is None:
        return None
    if p == 0 or r == 0:
        return 0
    return 1.0 / (alpha / p + (1 - alpha) / r)


### 1.4 Prompt format/ examples

In [4]:
svamp_format = """[INST] «SYS»\n
Given an arithmetic question, generate thoughts about the question step by step and then only give the answer as a number. Please follow the format below:

Thoughts:
<Step by Step Thoughts>
Answer:
<Number>

\n«/SYS»
Qestion:
{}
Thoughts:[/INST]"""

In [5]:
svamp_format.format('test')

'[INST] «SYS»\n\nGiven an arithmetic question, generate thoughts about the question step by step and then only give the answer as a number. Please follow the format below:\n\nThoughts:\n<Step by Step Thoughts>\nAnswer:\n<Number>\n\n\n«/SYS»\nQestion:\ntest\nThoughts:[/INST]'

### 1.5 Dataset

In [6]:
def build_test_dataset(
    dataset_name="/root/autodl-tmp/SVAMP",prompt_examples=svamp_format,prompt_format='default'
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
        prompt_examples ('str'):
            The few shot examples for testing
        prompt_format ('str'):
            The format of the prompting for testing

    Returns:
        dataset for test of checkpoints of llms
    """


    ds_test = load_dataset(dataset_name, split="test")
    original_columns = ds_test.column_names


    def preprocess_function(examples):

        body=examples["Body"]
        question=examples["Question"]
        if prompt_format=='default':
            query = svamp_format.format( body +' '+ question )

        return {"query": query,"full_question":body+' '+question }
    

    ds_test = ds_test.map(
        preprocess_function,
        batched=False,
        #remove_columns=original_columns,
    )


    print(ds_test)
    return ds_test

In [7]:
# We retrieve the dataloader by calling the `build_dataset` function.
dataset = build_test_dataset(dataset_name="/root/autodl-tmp/SVAMP")
prompt_test=dataset["query"]
questions_test=dataset["full_question"]
answer_test=dataset["Answer"]

HF google storage unreachable. Downloading and preparing it from source


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['Equation', 'Answer', 'Type', 'Body', 'Question', 'ID', 'query', 'full_question'],
    num_rows: 300
})


In [9]:
prompt_test[1]

'[INST] «SYS»\n\nGiven an arithmetic question, generate thoughts about the question step by step and then only give the answer as a number. Please follow the format below:\n\nThoughts:\n<Step by Step Thoughts>\nAnswer:\n<Number>\n\n\n«/SYS»\nQestion:\nPaul got a box of some crayons for his birthday. During the school year he gave 52 crayons to his friends while he lost 535 crayons. If he only had 492 crayons left How many crayons had been lost or given away?\nThoughts:[/INST]'

### 1.6 Function to Process Responses

In [53]:
ANS_RE = re.compile(r"Answer: .*?(\$?)(\-?[0-9\.\,]+)")
ANS_RE =re.compile(r"Answer:.*?(\$?)(\-?[0-9\.\,]+)", re.DOTALL)

INVALID_ANS = "[invalid]"
# extract the numerical answer for arithmetic dataset of the default prompt format
def _extract_answer(completion: str):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(2).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS



### 1.7 Tokenizer

In [10]:
# tokenizer for llama
model_name='/root/autodl-tmp/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### 1.8 Generate responses to questions

In [13]:
"""
def generate_responses(model,tokenizer,prompt_test=prompt_test,answer_test=answer_test,test_epochs=1,batch_size=2):


    # because of sampling of tokens, we can increase epochs of test for more precise evaluation
    test_epochs=test_epochs

    outputs_text=[]
    # the prompt and answer correspondong to output_text
    prompts_for_outputs=[]
    answers_for_outputs=[]

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_questions = prompt_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_inputs = tokenizer(batch_questions, padding=True, return_tensors="pt", truncation=True, max_length=2048).to('cuda')

            batch_outputs = model.generate(**batch_inputs,    
                                     max_new_tokens=256,
                                     top_k=50,
                                     temperature=1.0,
                                     top_p=0.95,
                                     do_sample=True,
                                     repetition_penalty=1.1)
            batch_input_length = batch_inputs.input_ids.shape[1]
            batch_generated_tokens = batch_outputs[:, batch_input_length:]
            batch_output_text = tokenizer.batch_decode(batch_generated_tokens)
            print(batch_output_text)
        
        
            outputs_text.extend(batch_output_text)
            prompts_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs


outputs=generate_responses(model=base_model,tokenizer=tokenizer)
"""

In [26]:
def generate_responses(generator,prompt_test=prompt_test,answer_test=answer_test,questions_test=questions_test,test_epochs=1,batch_size=4):
    generator=generator

    # if using sampling of tokens, we can increase epochs of test for more precise evaluation(law of large numbers)
    test_epochs=test_epochs

    outputs=[]
    prompts_for_outputs=[]
    questions_for_outputs=[]
    answers_for_outputs=[]
    

    batch_size = batch_size  

    for epoch in tqdm(range(test_epochs)):
        for i in tqdm(range(0, len(prompt_test), batch_size)):  # Increment by batch_size

            # Slice the data to create a batch
            batch_prompts = prompt_test[i:i+batch_size]
            batch_questions=questions_test[i:i+batch_size]
            batch_answers = answer_test[i:i+batch_size]

            batch_generations = generator(batch_prompts,batch_size=batch_size,return_full_text=False)
            print(batch_generations)
            outputs.extend([gen[0]['generated_text'] for gen in batch_generations])
            prompts_for_outputs.extend(batch_prompts)
            questions_for_outputs.extend(batch_questions)
            answers_for_outputs.extend(batch_answers)
    return outputs,questions_for_outputs,answers_for_outputs,prompts_for_outputs

# 2.base model

### 2.1 Load base model and builld pipeline

In [11]:
# load base model
model_name='/root/autodl-tmp/Llama-2-7b-chat-hf'
base_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                  torch_dtype=torch.float16,
                                                  device_map=0,
                                                 )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [16]:
gen_kwargs=dict(
    max_new_tokens=256,
    top_k=50,
    temperature=1.0,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1
)


# Create the pipeline for base model. To make results comparable, we use greedy search to eliminate the influence of sampling
base_generator= pipeline("text-generation", model=base_model, tokenizer=tokenizer,
                         max_new_tokens=512,
                         #top_k=50,
                         temperature=1.0,
                         #top_p=0.95,
                         do_sample=False,
                         #repetition_penalty=1.1,
                         #device=0,
                        )

### 2.2 Generate reponses by base model

In [27]:
base_responses,base_response_questions,base_response_answers,base_prompts=generate_responses(base_generator,batch_size=10)

  3%|▎         | 1/30 [00:09<04:42,  9.75s/it]

[[{'generated_text': "  Sure, I'd be happy to help! Here are my thoughts on the question:\nThoughts:\n1. Let's start by identifying the given information:\n* Mary has already added 5 cups of flour.\n* The recipe calls for 6 cups of flour.\n* The recipe also calls for 8 cups of sugar.\n* The recipe calls for 7 cups of salt.\n2. Now, we need to find out how many more cups of sugar Mary needs to add than cups of salt. Let's do this by subtracting the amount of flour she has already added from the total amount of flour needed:\n6 cups of flour - 5 cups of flour = 1 cup of flour remaining\nNow, we need to find out how many cups of sugar are left to add after subtracting the amount of flour remaining from the total amount of sugar needed:\n8 cups of sugar - 1 cup of flour = 7 cups of sugar remaining\n3. Finally, we can see that Mary needs to add 7 cups of sugar more than cups of salt to complete the recipe.\nAnswer: 7\n\nI hope this helps! Let me know if you have any questions."}], [{'genera


  7%|▋         | 2/30 [00:18<04:08,  8.87s/it]

[[{'generated_text': '\nThoughts:\n\n* Step 1: Identify the given information: Lewis earns $403 per week during the 233 weeks of harvest, and he has to pay $49 rent every week.\n* Step 2: Calculate the total amount of money Lewis earns during harvest season: 233 weeks x $403 per week = $93,530.\n* Step 3: Subtract the amount of rent Lewis has to pay each week: $93,530 - $49 per week = $89,530.\nAnswer: $89,530.'}], [{'generated_text': "\nThoughts:\nLet's start by identifying the information given in the question:\n\n* Jessie weighed 92 kilograms initially.\n* She lost 56 kilograms in the first week.\n* She lost 99 kilograms in the second week.\n\nNow, let's calculate Jessie's weight after the first week of jogging:\n\n* Initial weight = 92 kg\n* Loss in first week = 56 kg\n* New weight after first week = Initial weight - Loss in first week = 92 kg - 56 kg = 36 kg\n\nSo, Jessie weighed 36 kilograms after the first week of jogging.\nAnswer: 36 kg"}], [{'generated_text': '\nThoughts:\n\n1


 10%|█         | 3/30 [00:25<03:36,  8.02s/it]

...

[[{'generated_text': "\nThoughts:\n\n1. Let's start by identifying the information given in the question:\n\t* Mom buys 28 packages.\n\t* Each package contains 56 white t-shirts in total.\n2. To find out how many white t-shirts each package has, we need to divide the total number of t-shirts in each package by the number of packages:\n\t* 56 white t-shirts / 28 packages =?\n3. Now, let's simplify the fraction:\n\t* 56 / 28 = 2\n4. So, each package has 2 white t-shirts.\nAnswer: 2"}], [{'generated_text': '\nThoughts:\n\n1. Dan spent $ 7 on the candy bar.\n2. Dan spent $ 6 on the chocolate.\n3. Dan spent a total of $ 13 on the candy bar and chocolate.\nAnswer: $ 13'}], [{'generated_text': '\nThoughts:\n\n1. Danny had 25 bottle caps in his collection earlier.\n2. He found 32 bottle caps at the park.\n3. To find out how many more bottle caps he found, we need to subtract the number he already had from the number he found.\n4. So, the number of bottle caps he found at the park is 32 - 25 = 


 97%|█████████▋| 29/30 [04:36<00:09,  9.32s/it]

[[{'generated_text': '\nThoughts:\n\n1. Dan spent $2 to buy a candy bar.\n2. He spent $3 to buy a chocolate.\n3. So, the total amount he spent on both is $2 + $3 = $5.\n4. Therefore, he spent more money to buy the chocolate than he did to buy the candy bar.\nAnswer: $3'}], [{'generated_text': "\nThoughts:\n\n1. Let's start by identifying the information given in the question:\n\t* Tiffany had 7 bags of cans on Monday.\n\t* The next day, she found 12 more bags worth of cans.\n2. Let's convert the number of bags of cans from Monday to a numerical value:\n\t* 7 bags = 7 x 50 = 350 cans (since each bag contains 50 cans)\n3. Now, let's add the number of bags of cans Tiffany found on the next day to the number of bags she had on Monday:\n\t* 350 + 12 = 362 cans\n4. Therefore, Tiffany found 12 more bags of cans on the next day than she had on Monday.\nAnswer: 12"}], [{'generated_text': "\nThoughts:\nLet's start by identifying the information given in the question:\n\n* There are some baskets 


100%|██████████| 1/1 [04:47<00:00, 287.02s/it]

[[{'generated_text': "\nThoughts:\n\n1. Let's start by counting the total number of bottles of both regular and diet soda: 22 + 61 = 83 bottles.\n2. Now, let's subtract the number of regular soda bottles from the total number of bottles: 83 - 22 = 61 bottles.\n3. Therefore, the grocery store had 61 more bottles of diet soda than regular soda.\nAnswer: 61"}], [{'generated_text': "\nThoughts:\nLet's start by using the information we know:\n* Paul had 551 crayons at some point during the school year.\n* He lost or gave away 551 crayons.\n* He only had 177 crayons left at the end of the school year.\nNow, let's think about how we can use this information to find out how many crayons Paul got for his birthday:\n* If Paul had 551 crayons at some point during the school year, and he lost or gave away 551 crayons, then he must have had 0 crayons at the beginning of the school year.\n* So, the number of crayons Paul got for his birthday is equal to the number of crayons he had at the beginning 

### 2.3 Process Responses for base_model

In [55]:
base_extracted_responses = [_extract_answer(response) for response in base_responses]

print('invalid rate:',sum(np.array(base_extracted_responses)=='[invalid]')/len(base_extracted_responses))
np.array(base_responses)[np.array(base_extracted_responses)=='[invalid]']
#extracted_responses
#_extract_answer("Answer: $-12.1333,1 and 2,2")

invalid rate: 0.0


array([], dtype='<U1360')

### 2.4 Record the results for Base Model

In [41]:
import pandas as pd

# Create a DataFrame with named columns
df_base_svamp = pd.DataFrame({'Prompt':base_prompts,'Question': base_response_questions, 'Response': base_responses,'extracted_response':base_extracted_responses, 'Answer': base_response_answers})

# Write to CSV
df_base_svamp.to_csv('test_zeroshot_base_svamp.csv', index=True)

### 2.5 calculate metrics for base model

In [56]:
svamp_base_accuracy=accuracy(reference=base_response_answers, test=base_extracted_responses)

In [57]:
svamp_base_accuracy

0.4533333333333333

# 3.fine-tuned model

In [8]:
# load fine-tuned model
new_model_name="/root/autodl-tmp/msc_ml/llama_2/ckpts_svamp/checkpoint_3000"
new_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 torch_dtype=torch.float16,
                                                 device_map=0,
                                                )

new_model = PeftModel.from_pretrained(new_model, new_model_name)
new_model = new_model.merge_and_unload()

#model.add_adapter(lora_config, adapter_name="adapter_1")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'base_model' is not defined

In [9]:
# Create the pipeline for new model
new_generator= pipeline("text-generation", model=new_model, tokenizer=tokenizer,
                         max_new_tokens=512,
                         #top_k=50,
                         temperature=1.0,
                         #top_p=0.95,
                         do_sample=False,
                         #repetition_penalty=1.1,
                         #device=0,
                       )

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [50]:
svamp_3000_accuracy=accuracy(reference=extracted_responses, test=response_answers)
svamp_3000_accuracy 

NameError: name 'response_answers' is not defined

In [13]:
import rich
from rich.table import Table
rich_table = Table(*["prompt"], title='1', show_lines=True)
for ix in range(2):
    rich_table.add_row("""[/INST][INST]""")
rich_table

In [44]:
ds_all = load_dataset("/root/autodl-tmp/SVAMP", split="all")

In [45]:
ds_train = load_dataset("/root/autodl-tmp/SVAMP", split="train")

In [48]:
sum(np.array(ds_train['Body'])==np.array(ds_all['Body'][:700]))

700

In [61]:
ds_train['Answer']

[145.0,
 19.0,
 3.0,
 198.0,
 63.0,
 322.0,
 30.0,
 192.0,
 26.0,
 17.0,
 1.0,
 20.0,
 4.0,
 2.0,
 7.0,
 70.0,
 345.0,
 266.0,
 54.0,
 807.0,
 3.0,
 12.0,
 23.0,
 31.0,
 1201565.0,
 89.0,
 574.0,
 8.0,
 146.0,
 1.0,
 14.0,
 2.0,
 7.0,
 12.0,
 21.0,
 14.0,
 2.0,
 51.0,
 2.0,
 15.0,
 20.0,
 3.0,
 7.0,
 42.0,
 756.0,
 11.0,
 5.0,
 12.0,
 65.0,
 1.0,
 111.0,
 1.0,
 27.0,
 4.0,
 17.0,
 13.0,
 10.0,
 77.0,
 1092.0,
 5.0,
 7.0,
 45.0,
 10.0,
 52.0,
 2.0,
 6.0,
 16.0,
 469.0,
 12518.0,
 1.0,
 2.0,
 131.0,
 92.0,
 640.0,
 42.0,
 7.0,
 25.0,
 1.0,
 2.0,
 11.0,
 15.0,
 20.0,
 8722.0,
 5.0,
 83.0,
 29.0,
 53.0,
 3.0,
 33.0,
 1.0,
 16.0,
 47.0,
 5590.0,
 1.0,
 1.0,
 49.0,
 9.0,
 78.0,
 34.0,
 7.0,
 24.0,
 34.0,
 10.0,
 14.0,
 10.0,
 15.0,
 621.0,
 1.0,
 2.0,
 14.0,
 8.0,
 2.0,
 14.0,
 89.0,
 20.0,
 7.0,
 1.0,
 16.0,
 21.0,
 3.0,
 5.0,
 9.0,
 76.0,
 28.0,
 30.0,
 19.0,
 2.0,
 22.0,
 7.0,
 42.0,
 14.0,
 24.0,
 8.0,
 27.0,
 31.0,
 2.0,
 21.0,
 4.0,
 80.0,
 3.0,
 2.0,
 7.0,
 1.0,
 4.0,
 4.0,
 32.0,
 60